In [2]:
!pip install openai tiktoken python-dotenv requests

   ---------------------------------------- 0.0/883.8 kB ? eta -:--:--
   ---------------------------------------- 883.8/883.8 kB 5.5 MB/s eta 0:00:00


In [3]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# A Few Shot Learning

In [4]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

😍,✨,👏,🎉,🚀


# Prompt Engineering Best Practices

## Write clear instructions

Examples:

-----------------------
Prompt:

Write code to calculate the Fibonacci sequence.

Better:

Write a TypeScript function to efficiently calculate the Fibonacci sequence. Comment the code liberally to explain what each piece does and why it's written that way.

----------------------

Prompt:

Summarize the meeting notes.

Better:

Summarize the meeting notes in a single paragraph. Then write a markdown list of the speakers and each of their key points. Finally, list the next steps or action items suggested by the speakers, if any.


## Role Playing

Examples:

-----------------------

System Message: When I ask for help to write something, you will reply with a document that contains at least one joke or playful comment in every paragraph.

----------------------


## Segment input text

Examples:

------------------------

user message: Summarize the text delimited by triple quotes with a haiku.

"""insert text here"""

------------------------

system message: You will be provided with a pair of articles (delimited with XML tags) about the same topic. First summarize the arguments of each article. Then indicate which of them makes a better argument and explain why.

user message: 

\<article> insert first article here \</article>

\<article> insert second article here \</article>

------------------------

## Explain steps and processes to complete a task

Examples:

--------------------------------

System Message:
Use the following step-by-step instructions to respond to user inputs.

Step 1 - The user will provide you with text in triple quotes. Summarize this text in one sentence with a prefix that says "Summary: ".

Step 2 - Translate the summary from Step 1 into Spanish, with a prefix that says "Translation: ".

---------------------------------

## Ejercicio 1: Clasificación de Emojis con Zero-shot
Enunciado:
Dado el siguiente texto, predice hasta 5 emojis que mejor representen la emoción o el tema del mensaje. Usa el modelo de Zero-shot para hacer la clasificación.

Texto: "Estoy tan feliz que no puedo dejar de sonreír."

Resultado esperado: 😄,😊,✨,🌞,❤️

In [5]:
system_prompt = "Dado el siguiente texto, predice hasta 5 emojis que mejor representen la emoción o el tema del mensaje. Usa el modelo de Zero-shot para hacer la clasificación."
user_prompt = "Estoy tan feliz que no puedo dejar de sonreír."
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)




😊🎉💖😁🌟


## Ejercicio 2: Traducción de Resumen
Enunciado:
El siguiente texto es una transcripción de una reunión. Resúmelo en una sola oración, luego traduce ese resumen al inglés.

Texto:
"""Hoy discutimos sobre la necesidad de mejorar la interfaz de usuario en el sitio web. Los participantes estuvieron de acuerdo en que se debe hacer más intuitiva y accesible, especialmente para usuarios mayores. También se mencionó la importancia de agregar soporte multilingüe."""



In [6]:
system_prompt = "El siguiente texto es una transcripción de una reunión. Resúmelo en una sola oración"
user_prompt = """Hoy discutimos sobre la necesidad de mejorar la interfaz de usuario en el sitio web. Los participantes estuvieron de acuerdo en que se debe hacer más intuitiva y accesible, especialmente para usuarios mayores. También se mencionó la importancia de agregar soporte multilingüe."""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt}])
summary = response.choices[0].message.content
print(f"Summary: {summary}")

# Traducción al inglés
translation_prompt = f"Traduce el siguiente resumen al Ingles: {summary}"
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [{"role":"system", "content": "Eres un traductor"},
                {"role":"user","content": translation_prompt}])
print(f"Translation: {response.choices[0].message.content}")


Summary: Se llegó a un consenso sobre la necesidad de mejorar la interfaz de usuario del sitio web para que sea más intuitiva y accesible, especialmente para usuarios mayores, así como la inclusión de soporte multilingüe.
Translation: A consensus was reached on the need to improve the website's user interface to make it more intuitive and accessible, especially for older users, as well as the inclusion of multilingual support.


Ejemplo de salida:

Summary: Se discutió la necesidad de mejorar la interfaz de usuario del sitio web para que sea más intuitiva y accesible, especialmente para usuarios mayores, y se destacó la importancia de incluir soporte multilingüe.  
Today we discussed the need to improve the website's user interface to make it more intuitive and accessible, especially for older users, and emphasized the importance of adding multilingual support.
Translation: The following summary is translated into English: 

"We discussed the need to improve the website's user interface to make it more intuitive and accessible, especially for older users, and emphasized the importance of including multilingual support."

## Ejercicio 4: Respuesta Automática con Role-playing
Enunciado:
Configura un modelo para responder a una solicitud de escritura, haciendo que la respuesta contenga un toque de humor. Usa la siguiente solicitud como ejemplo.

Solicitud:
"Escribe una carta formal solicitando días de vacaciones."



**Salida esperada:**

[Tu nombre]  
[Tu dirección]  
[Ciudad, Estado, Código Postal]  
[Correo electrónico]  
[Teléfono]  
[Fecha]  

[Nombre del jefe o supervisor]  
[Nombre de la empresa]  
[Dirección de la empresa]  
[Ciudad, Estado, Código Postal]  

Estimado/a [Nombre del jefe o supervisor]:

Espero que este mensaje lo/a encuentre en un buen estado de ánimo y que el café de la mañana aún esté dando su efecto. Me dirijo a usted para solicitar formalmente unos días de vacaciones debido a la necesidad de recargar pilas y asegurarme de que mi creatividad no se convierta en un documento en blanco. Después de todo, un buen descanso es tan esencial como un buen chiste para iniciar la jornada, ¿verdad?

Me gustaría solicitar [número de días] días de vacaciones, del [fecha de inicio] al [fecha de finalización]. Estoy seguro/a de que puedo entregar todas mis responsabilidades y proyectos antes de esta fecha, para que el equipo no se vea afectado. Prometo no dejar una nota que diga “En caso de emergencia, llame a mi gato”, ¡aunque sé que muchos lo harían por gracia!

Agradezco de antemano su comprensión y apoyo en esta solicitud. Estoy dispuesto/a a discutir cualquier detalle adicional y a asegurarme de que la transición sea lo más fluida posible. Al fin y al cabo, no quiero que mis colegas piensen que huyo del trabajo, aunque una playa y un cóctel pueden sonar bastante tentadores.

Quedo a la espera de su pronta respuesta. ¡Prometo volver con muchas historias, y quizás algunos souvenirs divertidos (o un sandalia como souvenir) para compartir!

Atentamente,  
[Tu nombre]  
[Tu puesto]  

In [7]:
system_prompt = "Cuando te pida ayuda para escribir algo, me responderás con un documento que contenga al menos un chiste o comentario gracioso en cada párrafo."
user_prompt = "Escribe una carta formal solicitando días de vacaciones."

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: carta con broma


[Tu Nombre]  
[Tu Dirección]  
[Ciudad, Estado, Código Postal]  
[Tu Correo Electrónico]  
[Tu Teléfono]  
[Fecha]  

[Nombre del Jefe]  
[Nombre de la Empresa]  
[Dirección de la Empresa]  
[Ciudad, Estado, Código Postal]  

Estimado/a [Nombre del Jefe]:

Espero que este mensaje lo encuentre bien y de buen humor. ¡Si no es así, puedo ofrecerte una broma sobre por qué los pájaros no usan Facebook... porque ya tienen Twitter! Pero, pasando a lo que realmente importa, me gustaría solicitar formalmente unos días de vacaciones.

Conforme a las políticas de la empresa, quisiera solicitar [número de días] días de vacaciones desde el [fecha de inicio] hasta el [fecha de finalización]. La razón de mi solicitud es que, después de un tiempo intensivo de trabajo, es importante recargar energías... como cuando se resetea el WiFi, ¡es un verdadero salvavidas! Estoy seguro/a de que regresaré con más motivación y un par de anécdotas graciosas para compartir.

Durante mi ausencia, me aseguraré de que 

## Ejercicio 5: Segmentación de Texto
Enunciado:
Segmenta un texto delimitado por comillas triples y luego tradúcelo a otro idioma (por ejemplo, inglés). Aplica el siguiente formato de entrada de texto y muestra cómo lo harías.

Texto de Entrada:
"""Hoy es un buen día para aprender nuevas cosas y mejorar nuestras habilidades. ¡El futuro está lleno de oportunidades!"""


#### Ejemplo de salida:

Summary: Hoy es un buen día para aprender y mejorar habilidades, ya que el futuro está lleno de oportunidades.  
Translation: Today is a good day to learn and improve skills, as the future is full of opportunities.

In [9]:
system_prompt = "Resume el texto delimitado por comillas triples y escribe la traducción en inglés."

user_prompt = """Hoy es un buen día para aprender nuevas cosas y mejorar nuestras habilidades. ¡El futuro está lleno de oportunidades!"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Salida esperada: resumen en español e inglés


Resumen: Hoy es un excelente momento para adquirir conocimientos y desarrollar habilidades, ya que el futuro ofrece muchas oportunidades.

Translation: Today is a great time to acquire knowledge and develop skills, as the future offers many opportunities.


## Ejercicio 6: Predicción de Emojis con Zero-shot
Enunciado:
Realiza una clasificación Zero-shot utilizando el modelo y predice hasta 5 emojis para el siguiente mensaje:

Texto de entrada:
"Estoy muy cansado, pero contento con el trabajo que he hecho."



In [ ]:
system_prompt = """Predice hasta 5 emojis como respuesta a un mensaje de chat. La salida debe solo incluir emojis.
"""
user_prompt = "1.Estoy muy cansado, pero contento con el trabajo que he hecho. \\n\\
2.Estoy contento y motivado estudiando IA. "
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: "😅,💪,👌"

💻✨😌💪🌟


## Ejercicio 7: Predicción de Emojis con Few-shot Learning

Enunciado:

En este ejercicio utilizarás Few-shot learning con roles (system, user, assistant) para predecir emojis. En Few-shot learning, proporcionamos ejemplos específicos de interacciones para que el modelo aprenda a generalizar la tarea con pocos ejemplos.

Dado un conjunto de ejemplos de interacciones entre un usuario y un asistente, utiliza el modelo para predecir hasta 5 emojis para el siguiente mensaje:

Mensaje: "Estoy muy cansado, pero contento con el trabajo que he hecho."

Devuelve solo los emojis, separados por comas.

In [73]:
# Definir el prompt con Few-shot utilizando múltiples roles
few_shot_prompt = [
    {"role": "system", "content": "You are an assistant that predicts emojis based on user text."},
    
    {"role": "user", "content": "Estoy muy feliz con mi nuevo trabajo."},
    {"role": "assistant", "content": "😊,🎉,💼,💪"},
    
    {"role": "user", "content": "No me siento bien, tengo dolor de cabeza."},
    {"role": "assistant", "content": "😞,🤕,💔"},
    
    {"role": "user", "content": "Hoy es un día increíble para correr al aire libre."},
    {"role": "assistant", "content": "🏃‍♂️,🌳,🌞,💨"},
    
    {"role": "user", "content": "Acabo de terminar de ver una película emocionante."},
    {"role": "assistant", "content": "🎬,👏,😱,❤️"},
    
    {"role": "user", "content": "Voy a dormir temprano, me siento agotado."},
    {"role": "assistant", "content": "😴,🛏️,😴"},
    
    # Ahora pedimos la predicción para el nuevo mensaje
    {"role": "user", "content": "Estoy muy cansado, pero contento con el trabajo que he hecho."},
]

# Llamada a la API para obtener la predicción de emojis
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=few_shot_prompt
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: "😅,💪,👌"

😌,💼,👏,😴
